

<h1><center><strong>Missing Value Treatment</strong></center></h1>

# **Table of Contents**

1. [Diabetes Dataset](#Section1)<br><br>
2. [Marking Missing Values](#Section2)<br><br>
3. [Remove Rows With Missing Values](#Section3)<br><br>
4. [Fill Missing Values with a Test Stastic](#Section4)<br><br>
5. [Predict Missing Values With a Machine Learning Algorithm](#Section5)<br><br>
6. [Conclusion](#Section6)

<a id = Section1></a>
### **1. Diabetes Dataset**

- The Diabetes Dataset involves **predicting** the **onset of diabetes** within 5 years in given medical details.

- It is a **binary** (**2-class**) classification problem.

- The number of observations for each class is not balanced.

- There are **768 observations** with **8 input** variables and **1 output** variable.

- The variable names are as follows:

    **0**. Number of times pregnant.

    **1**. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.

    **2**. Diastolic blood pressure (mm Hg).

    **3**. Triceps skinfold thickness (mm).

    **4**. 2-Hour serum insulin (mu U/ml).

    **5**. Body mass index (weight in kg/(height in m)^2).

    **6**. Diabetes pedigree function.

    **7**. Age (years).

    **8**. Class variable (0 or 1).

- This dataset is known to have **missing values**.

- Specifically, there are missing observations for some columns that are **marked** as a **zero** value.

- We can corroborate this by the definition of those columns and the domain knowledge that a **zero value** is **invalid** for those measures, e.g. a zero for body mass index or blood pressure is invalid.

#### **Importing the Dataset**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
missing_df = pd.read_csv('https://raw.githubusercontent.com/insaid2018/Term-2/master/Data/pima-indians-diabetes.csv', header=None)
missing_df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
missing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


<a id = Section2></a>
### **2. Marking Missing Values**

In [ ]:
missing_df.describe()

,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


- We can see that there are columns that have a **minimum** value of **zero** (**0**).

- On some columns, a value of **zero** does not make sense and **indicates** an **invalid** or **missing** value.

- Specifically, the following columns have an **invalid zero minimum** value:

    **1**. Plasma glucose concentration

    **2**. Diastolic blood pressure
    
    **3**. Triceps skinfold thickness
    
    **4**. 2-Hour serum insulin
    
    **5**. Body mass index

- Let’s confirm this my looking at the raw data, the example prints the first 20 rows of data.

In [ ]:
missing_df.head(20)

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


- We can get a **count** of the **number** of **missing** values on each of these columns.

- We can do this my marking all of the values in the subset of the DataFrame we are interested in that have zero values as True.

- We can then count the number of true values in each column.

In [ ]:
# Count the number of missing values for each column
num_missing = (missing_df[[1,2,3,4,5]] == 0).sum()

In [ ]:
# Report the results
print(num_missing)

1      5
2     35
3    227
4    374
5     11
dtype: int64


- We can see that columns **1**, **2** and **5** have just a **few zero values**, whereas columns **3** and **4** show a lot more, nearly **half** of the **rows**.

- This highlights that **different “missing value” strategies** may be **needed** for different columns, e.g. to ensure that there are still a sufficient number of records left to train a predictive model.

# New Section

- In Python, specifically Pandas, NumPy and Scikit-Learn, we mark **missing values** as **NaN**.

- Values with a NaN value are **ignored** from operations like **sum**, **count**, etc.

- We can **mark** values as **NaN** easily with the Pandas DataFrame by using the `replace()` function on a subset of the columns we are interested in.

- After we have marked the missing values, we can use the `isnull()` function to **mark** all of the **NaN** values in the dataset as **True** and get a **count** of the missing values for each column.

In [ ]:
# Replace '0' values with 'nan'
missing_df[[1,2,3,4,5]] = missing_df[[1,2,3,4,5]].replace(0, np.nan)

In [ ]:
# Count the number of nan values in each column
print(missing_df.isnull().sum())

0      0
1      5
2     35
3    227
4    374
5     11
6      0
7      0
8      0
dtype: int64


- Running the above code prints the **number of missing values** in **each column**.

- We can see that the columns 1:5 have the same number of missing values as zero values identified above.

- This is a sign that we have marked the identified missing values correctly.

In [ ]:
missing_df.head(20)

,0,1,2,3,4,5,6,7,8
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
5,5,116.0,74.0,NaN,NaN,25.6,0.201,30,0
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
7,10,115.0,NaN,NaN,NaN,35.3,0.134,29,0
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
9,8,125.0,96.0,NaN,NaN,NaN,0.232,54,1


- It is clear from the raw data that marking the missing values had the intended effect.

#### **Missing Values Causes Problems**

- Having missing values in a dataset can **cause errors** with some machine learning algorithms.

- Most predictive modeling techniques **cannot handle** any **missing values**.

- Therefore, this problem must be **addressed prior** to **modeling**.

- Now, we can look at methods to handle the missing values.

<a id = Section3></a>
### **3. Remove Rows With Missing Values**

- The simplest strategy for handling missing data is to **remove records** that **contain** a **missing value**.

- We can do this by creating a new Pandas DataFrame with the rows containing missing values removed.

- Pandas provides the `dropna()` function that can be used to **drop either columns or rows** with missing data.

- We can use dropna() to remove all rows with missing data, as follows:

In [ ]:
# Count the number of nan values in each column
print(missing_df.isnull().sum())

0      0
1      5
2     35
3    227
4    374
5     11
6      0
7      0
8      0
dtype: int64


In [ ]:
# Drop rows with missing values
reduced_df = missing_df.dropna()

In [ ]:
# Summarize the shape of the data with missing rows removed
print(missing_df.shape)
print(reduced_df.shape)

(768, 9)
(392, 9)


- After running this code, we can see that the number of rows has been aggressively cut from **768** in the original dataset to **392** with all rows containing a NaN removed.

- But **removing rows** with missing values can be **too limiting** on some predictive modeling problems, and an alternative is to impute missing values.

#### **Training a ML model with the Reduced Dataframe**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
clf = LogisticRegression(random_state=0)

In [ ]:
# Evaluate the model
scores = cross_val_score(clf, reduced_df.iloc[:, :-1], reduced_df.iloc[:, -1], cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
# Report the mean performance
print('Accuracy: %.3f' % scores.mean())

Accuracy: 0.771


- We get an **accuracy** of **77.1%** after **removing all** the **rows** containing **missing** values.

<a id = Section4></a>
### **4. Fill Missing Values with a Test Statistic**

- **Imputing** refers to using a model to **replace missing values**.

- There are many options we could consider when replacing a missing value, for example:

  - A **constant value** that has meaning within the domain, such as 0, distinct from all other values.

  - A value from another randomly selected record.

  - A **mean**, **median** or **mode** value for the column.

  - A value estimated by another predictive model.

#### **Using fillna()**:

- Any **imputing** performed on the **training** dataset will have to be **performed** on **new data** in the future when predictions are needed from the finalized model. 

- This needs to be taken into consideration when choosing how to impute the missing values.

- For example, if you choose to impute with mean column values, these mean column values will need to be stored to file for later use on new data that has missing values.

- Pandas provides the `fillna()` function for **replacing missing values** with a specific value.

- For example, we can use `fillna()` to **replace missing values** with the **mean** value for each column, as follows:

In [ ]:
# Count the number of nan values in each column
print(missing_df.isnull().sum())

0      0
1      5
2     35
3    227
4    374
5     11
6      0
7      0
8      0
dtype: int64


- You can **provide** the **value** you want to use to **fill** the missing values in the `fillna` function.

- Use `missing_df.median()` for feature **median** values, `missing_df.mode().iloc[0, :]` for feature **mode** values.

- Or, you can provide a **constant string** or **numeric** value to be used for filling the missing values.

In [ ]:
# Fill missing values with mean column values
mean_filled_df = missing_df.fillna(missing_df.mean())

In [ ]:
# Count the number of NaN values in each column
print(mean_filled_df.isnull().sum())

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64


- Running this code provides a count of the number of missing values in each column, showing **zero missing values**.

#### **Training a ML model with the Mean-Filled Dataframe**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
clf = LogisticRegression(random_state=0)

In [ ]:
# Evaluate the model
scores = cross_val_score(clf, mean_filled_df.iloc[:, :-1], mean_filled_df.iloc[:, -1], cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
# Report the mean performance
print('Accuracy: %.3f' % scores.mean())

Accuracy: 0.763


- We get an **accuracy** of **76.3%** after **filing** the **missing** values with **mean** of each column using `fillna()`.

#### **Using SimpleImputer**:

- The scikit-learn library provides the **SimpleImputer** pre-processing class that can be used to **replace missing values**.

- It is a flexible class that allows you to **specify** the **value** to **replace** (it can be something other than NaN) and the **technique** used to replace it (such as mean, median, or mode).

- The **SimpleImputer** class operates directly on the **NumPy array** instead of the DataFrame.

- The example below uses the SimpleImputer class to **replace missing values** with the **mean** of each column then prints the number of NaN values in the transformed matrix.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# Retrieve the numpy array
values = missing_df.values

In [ ]:
values

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [ ]:
# Count the number of NaN values in each column
print('Missing: %d' % np.isnan(values).sum())

Missing: 652


- You can **change** the `strategy` from **mean** to **median**, **most_frequent**, or **constant**, as per your requirements.

In [ ]:
# Define the imputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
# Transform the dataset
transformed_values = imputer.fit_transform(values)

In [ ]:
# Count the number of NaN values in each column
print('Missing: %d' % np.isnan(transformed_values).sum())

Missing: 0


- Running this code shows that **all NaN** values were **imputed** successfully.

#### **Training a ML model with the Mean-Imputed Array**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
clf = LogisticRegression(random_state=0)

In [ ]:
# Evaluate the model
scores = cross_val_score(clf, transformed_values[:, :-1], transformed_values[:, -1], cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
# Report the mean performance
print('Accuracy: %.3f' % scores.mean())

Accuracy: 0.766


- We get an **accuracy** of **76.6%** after **filing** the **missing** values with **mean** of each column using **SimpleImputer**.

<a id = Section5></a>
### **5. Predict Missing Values With a Machine Learning Algorithm**

**Process**:

1. **Call** the **variable** where you have **missing values** as **y**.

2. **Split data** into sets **with missing values** and **without missing values**.

  - **Name** the **missing set X_text** and the one **without missing values X_train**.
  
  - **Take y** (variable or feature where there is missing values) off the **second set**, naming it **y_train**.

3. **Use** one of **classification methods** to **predict y_pred**.

4. **Add** it to **X_test** as your **y_test** column.

5. Then **combine sets** together.

#### **Using Scikit-Learn's IterativeImputer**:

- The scikit-learn library provides the **IterativeImputer** class, which models each feature with missing values as a function of other features, and uses that estimate for imputation.

- It does so in an iterated round-robin fashion: at each step, a **feature** column is **designated** as **output y** and the **other feature columns** are treated as **inputs X**.

- A **regressor** is **fit** on **(X, y)** for **known y**.

- Then, the **regressor** is used to **predict** the **missing** values of **y**.

- This is done for **each feature** in an iterative fashion, and then is repeated for max_iter imputation rounds.

- The results of the final imputation round are returned.

**Note: This estimator is still experimental for now: default parameters or details of behaviour might change without any deprecation cycle.**

[IterativeImputer Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
# Retrieve the numpy array
values = missing_df.values

In [ ]:
values

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [ ]:
# Count the number of NaN values in each column
print('Missing: %d' % np.isnan(values).sum())

Missing: 652


In [ ]:
# Define the imputer
iterative_imp = IterativeImputer(max_iter=10, random_state=0)

In [ ]:
# Transform the dataset
transformed_values = iterative_imp.fit_transform(values)

In [ ]:
# Count the number of NaN values in each column
print('Missing: %d' % np.isnan(transformed_values).sum())

Missing: 0


- Running this code shows that **all NaN** values were **imputed** successfully.

#### **Training a ML model with the Iteratively-Imputed Array**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
clf = LogisticRegression(random_state=0)

In [ ]:
# Evaluate the model
scores = cross_val_score(clf, transformed_values[:, :-1], transformed_values[:, -1], cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
# Report the mean performance
print('Accuracy: %.3f' % scores.mean())

Accuracy: 0.758


- We get an **accuracy** of **75.8%** after **filing** the **missing** values using **IterativeImputer**.

#### **Nearest Neighbors Imputation**

- The **KNNImputer** class provides imputation for filling in missing values using the k-Nearest Neighbors approach.

- By default, a euclidean distance metric that supports missing values, **nan_euclidean_distances**, is used to find the nearest neighbors.

- Each missing feature is imputed using values from **n_neighbors** nearest neighbors that have a value for the feature.

- The feature of the neighbors are **averaged uniformly** or **weighted by distance** to each neighbor.

- If a sample has more than one feature missing, then the neighbors for that sample can be different depending on the particular feature being imputed.

- If a feature is always missing in training, it is removed during transform.

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
# Retrieve the numpy array
values = missing_df.values

In [ ]:
values

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [ ]:
# Count the number of NaN values in each column
print('Missing: %d' % np.isnan(values).sum())

Missing: 652


In [ ]:
# Define the imputer
knn_imputer = KNNImputer(n_neighbors=3, weights="uniform")

In [ ]:
# Transform the dataset
transformed_values = knn_imputer.fit_transform(values)

In [ ]:
# Count the number of NaN values in each column
print('Missing: %d' % np.isnan(transformed_values).sum())

Missing: 0


- Running this code shows that all NaN values were imputed successfully.

#### **Training a ML model with the KNN-Imputed Array**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
clf = LogisticRegression(random_state=0)

In [ ]:
# Evaluate the model
scores = cross_val_score(clf, transformed_values[:, :-1], transformed_values[:, -1], cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
# Report the mean performance
print('Accuracy: %.3f' % scores.mean())

Accuracy: 0.770


- We get an **accuracy** of **77%** after **filing** the **missing** values using **KNNImputer**.

<a id = Section6></a>
### **6. Conclusion**

- We discussed **different methods** of **removing missing values** from the dataset.

- If the **dataset** is very **large**, and the **number** of **rows** containing **missing values** is **proportionally small**, we can **drop** the **rows** containing missing values.

- Most of the times, **filling** the **missing values** with a test statistic like mean, median or mode of the feature is the **best** way to remove missing values.

- Also, we can **use ML models** like **KNN** to fill missing values in the data as well.

- But, since **each dataset** is **different**, these methods need to be **tested iteratively** on each dataset and the **model performance** should be **compared** to find the best way of removing missing values from a dataset.

- In this case, **removing** the **rows** containing missing values turned out to be **better** than the other imputation methods we used.

  - But, we can also try to **fill** the missing values **using median** and **mode** of the features, as **mean** is **susceptible** to **outliers**.

  - Also, KNNImputer proved to be much better than IterativeImputer in this case, but it might change for a different dataset.

  - So, we should always **experiment** with **different** missing value treatment **methods** before finalizing one.